In [2]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from dotenv import load_dotenv
import datetime

In [3]:
load_dotenv()
jdbc_driver_path = "postgresql-42.7.4.jar"

In [4]:
spark = SparkSession.builder \
    .appName('Solution') \
    .config("spark.driver.extraClassPath", jdbc_driver_path) \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

In [5]:
jdbcHostname = os.getenv("HOST")
jdbcDatabase = os.getenv("DB_NAME")
jdbcUsername = os.getenv("USER")
jdbcPassword = os.getenv("PASSWORD")
jdbcPort = 5432 
jdbcDriver = "org.postgresql.Driver"

connProperties = {
  "user": jdbcUsername,
  "password": jdbcPassword,
  "driver": jdbcDriver
}

jdbcUrl = f"jdbc:postgresql://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"


In [6]:
bonus_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM bonus)", properties=connProperties)
title_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM title)", properties=connProperties)
worker_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM worker)", properties=connProperties)

#### Q-1. Fetch “FIRST_NAME” from Worker table using the alias name as WORKER_NAME.

In [7]:
worker_df.select(col('first_name').alias('WORKER_NAME')).show()

+-----------+
|WORKER_NAME|
+-----------+
|      Megan|
|  Katherine|
|     Robert|
|   Jonathan|
|    William|
|    Richard|
|    Kristen|
|      Kevin|
|     Thomas|
|     Brandy|
|    Rebecca|
|    William|
|       Juan|
|    Katelyn|
|  Christine|
|       John|
|      Renee|
|      Tonya|
|       Lisa|
|     Rachel|
+-----------+
only showing top 20 rows



#### Q-2. Fetch “FIRST_NAME” from Worker table in upper case.

In [8]:
worker_df.select(upper(col('first_name').alias('WORKER_NAME'))).show()

+--------------------------------+
|upper(first_name AS WORKER_NAME)|
+--------------------------------+
|                           MEGAN|
|                       KATHERINE|
|                          ROBERT|
|                        JONATHAN|
|                         WILLIAM|
|                         RICHARD|
|                         KRISTEN|
|                           KEVIN|
|                          THOMAS|
|                          BRANDY|
|                         REBECCA|
|                         WILLIAM|
|                            JUAN|
|                         KATELYN|
|                       CHRISTINE|
|                            JOHN|
|                           RENEE|
|                           TONYA|
|                            LISA|
|                          RACHEL|
+--------------------------------+
only showing top 20 rows



#### Q-3. Fetch unique values of DEPARTMENT from Worker table.

In [9]:
worker_df.select(col('department')).distinct().show()

+-----------+
| department|
+-----------+
|Engineering|
|         HR|
|      Admin|
|  Marketing|
|    Account|
+-----------+



#### Q-4. Print the first three characters of FIRST_NAME from Worker table.

In [10]:
worker_df.select(substring('first_name',1,3).alias('first_name')).show()

+----------+
|first_name|
+----------+
|       Meg|
|       Kat|
|       Rob|
|       Jon|
|       Wil|
|       Ric|
|       Kri|
|       Kev|
|       Tho|
|       Bra|
|       Reb|
|       Wil|
|       Jua|
|       Kat|
|       Chr|
|       Joh|
|       Ren|
|       Ton|
|       Lis|
|       Rac|
+----------+
only showing top 20 rows



#### Q-5. Find the position of the alphabet (‘b’) in the first name column ‘Amitabh’ from Worker table.

In [11]:
worker_df.withColumn("position_of_b", instr(col("first_name"), "b")) \
    .select(col('position_of_b')) \
    .show()

+-------------+
|position_of_b|
+-------------+
|            0|
|            0|
|            3|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            3|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
+-------------+
only showing top 20 rows



#### Q-6. Print the FIRST_NAME from Worker table after removing white spaces from the right side.

In [12]:
worker_df.withColumn('first_name',rtrim(col('first_name'))) \
        .select('first_name') \
        .show()

+----------+
|first_name|
+----------+
|     Megan|
| Katherine|
|    Robert|
|  Jonathan|
|   William|
|   Richard|
|   Kristen|
|     Kevin|
|    Thomas|
|    Brandy|
|   Rebecca|
|   William|
|      Juan|
|   Katelyn|
| Christine|
|      John|
|     Renee|
|     Tonya|
|      Lisa|
|    Rachel|
+----------+
only showing top 20 rows



#### Q-7. Print the DEPARTMENT from Worker table after removing white spaces from the left side.

In [13]:
worker_df.withColumn('department',ltrim(col('department'))) \
        .select(col('department')) \
        .show()

+-----------+
| department|
+-----------+
|      Admin|
|         HR|
|  Marketing|
|         HR|
|         HR|
|Engineering|
|    Account|
|      Admin|
|      Admin|
|    Account|
|Engineering|
|         HR|
|         HR|
|Engineering|
|Engineering|
|    Account|
|    Account|
|Engineering|
|  Marketing|
|    Account|
+-----------+
only showing top 20 rows



#### Q-8. Fetch the unique values of DEPARTMENT from Worker table and print its length.

In [14]:
worker_df.withColumn("length", length(col('department'))) \
    .select('department','length') \
    .distinct() \
    .show()

+-----------+------+
| department|length|
+-----------+------+
|         HR|     2|
|Engineering|    11|
|      Admin|     5|
|    Account|     7|
|  Marketing|     9|
+-----------+------+



#### Q-9. Print the FIRST_NAME from Worker table after replacing ‘a’ with ‘A’.

In [15]:
worker_df.withColumn('first_name',regexp_replace('first_name','a','A')) \
        .select('first_name') \
        .show()

+----------+
|first_name|
+----------+
|     MegAn|
| KAtherine|
|    Robert|
|  JonAthAn|
|   WilliAm|
|   RichArd|
|   Kristen|
|     Kevin|
|    ThomAs|
|    BrAndy|
|   RebeccA|
|   WilliAm|
|      JuAn|
|   KAtelyn|
| Christine|
|      John|
|     Renee|
|     TonyA|
|      LisA|
|    RAchel|
+----------+
only showing top 20 rows



#### Q-10. Print the FIRST_NAME and LAST_NAME from Worker table into a single column COMPLETE_NAME. A space char should separate them.

In [16]:
worker_df.withColumn('complete_name',concat(col('first_name'),lit(' '),col('last_name'))) \
    .select('complete_name') \
    .show()

+-----------------+
|    complete_name|
+-----------------+
|         Megan Li|
|  Katherine Mccoy|
|     Robert Banks|
|  Jonathan Miller|
|  William Elliott|
|       Richard Le|
|    Kristen Davis|
|     Kevin Thomas|
|    Thomas Harris|
|   Brandy Sanchez|
|     Rebecca Mack|
|William Rodriguez|
|      Juan Murray|
|   Katelyn Harris|
|   Christine Carr|
|      John Castro|
|    Renee Holland|
|     Tonya Osborn|
|      Lisa Foster|
|    Rachel Barnes|
+-----------------+
only showing top 20 rows



#### Q-11. Print all Worker details from the Worker table order by FIRST_NAME Ascending.

In [22]:
worker_df.orderBy(col('first_name').asc()).show()

+---------+----------+---------+------+------------+-----------+
|worker_id|first_name|last_name|salary|joining_date| department|
+---------+----------+---------+------+------------+-----------+
|     2955|     Aaron|   Butler|398344|     15:40.4|Engineering|
|     6928|     Aaron|   Guzman|212060|     37:50.6|      Admin|
|     3845|     Aaron|   Cooper|372431|     20:46.6|Engineering|
|     6906|     Aaron|   Hansen|176409|     54:17.1|  Marketing|
|     3968|     Aaron|     Hall|434826|     54:10.9|Engineering|
|      403|     Aaron|    Watts|379063|     04:33.5|  Marketing|
|     4917|     Aaron|   Keller|171408|     52:23.8|      Admin|
|      907|     Aaron|Middleton|316512|     43:00.4|    Account|
|     5080|     Aaron| Holloway|394001|     06:55.9|      Admin|
|     2704|     Aaron|    Ramos|115298|     00:59.7|  Marketing|
|     5468|     Aaron|   Powers|238662|     49:26.7|Engineering|
|     1550|     Aaron|  Hoffman|139818|     47:19.6|  Marketing|
|     5756|     Aaron|   

#### Q-12. Print all Worker details from the Worker table order by FIRST_NAME Ascending and DEPARTMENT Descending.

In [28]:
worker_df.orderBy(col('first_name').asc(),col('department').desc()).show()

+---------+----------+---------+------+------------+-----------+
|worker_id|first_name|last_name|salary|joining_date| department|
+---------+----------+---------+------+------------+-----------+
|     2608|     Aaron|    Brown| 75495|     14:39.2|  Marketing|
|     6325|     Aaron|   Duncan|414977|     51:49.0|  Marketing|
|     2704|     Aaron|    Ramos|115298|     00:59.7|  Marketing|
|      403|     Aaron|    Watts|379063|     04:33.5|  Marketing|
|     1550|     Aaron|  Hoffman|139818|     47:19.6|  Marketing|
|     6906|     Aaron|   Hansen|176409|     54:17.1|  Marketing|
|     8124|     Aaron|    Scott|201596|     25:51.3|  Marketing|
|     9835|     Aaron|  Elliott| 76718|     49:50.0|  Marketing|
|     1974|     Aaron|   Torres|341462|     17:24.9|         HR|
|      226|     Aaron| Williams|375373|     09:22.7|         HR|
|     9935|     Aaron|  Mueller|368324|     25:46.0|         HR|
|      187|     Aaron|    Brown|415780|     04:47.4|Engineering|
|      994|     Aaron|   

#### Q-13. Print details for Workers with the first name as “Vipul” and “Satish” from Worker table.

In [29]:
worker_df.filter(col('first_name').isin('Vipul', 'Satish')).show()

+---------+----------+---------+------+------------+----------+
|worker_id|first_name|last_name|salary|joining_date|department|
+---------+----------+---------+------+------------+----------+
+---------+----------+---------+------+------------+----------+



#### Q-14. Print details of workers excluding first names, “Vipul” and “Satish” from Worker table.

In [31]:
worker_df.filter(~col('first_name').isin('Vipul', 'Satish')).show()

+---------+----------+---------+------+------------+-----------+
|worker_id|first_name|last_name|salary|joining_date| department|
+---------+----------+---------+------+------------+-----------+
|        1|     Megan|       Li|492720|     26:31.7|      Admin|
|        2| Katherine|    Mccoy|251979|     47:06.7|         HR|
|        3|    Robert|    Banks|447386|     57:31.6|  Marketing|
|        4|  Jonathan|   Miller|270500|     11:21.1|         HR|
|        5|   William|  Elliott| 71225|     54:32.3|         HR|
|        6|   Richard|       Le|185746|     45:33.0|Engineering|
|        7|   Kristen|    Davis|318055|     36:46.6|    Account|
|        8|     Kevin|   Thomas|304766|     30:42.8|      Admin|
|        9|    Thomas|   Harris|262302|     22:34.8|      Admin|
|       10|    Brandy|  Sanchez|460936|     48:18.7|    Account|
|       11|   Rebecca|     Mack|485081|     20:15.9|Engineering|
|       12|   William|Rodriguez|209023|     13:13.3|         HR|
|       13|      Juan|   